In [6]:
import os
import json
import numpy as np
from datetime import datetime, timedelta
import time
from multiprocessing import Pool

from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimatorV2 as BackendEstimator

from susy_qm import calculate_Hamiltonian

In [3]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

In [47]:
noise_model = NoiseModel.from_backend(backend)

In [48]:
simulator = AerSimulator(
    noise_model=noise_model,
    #basis_gates=noise_model.basis_gates,
    #coupling_map=backend.configuration().coupling_map,
    shots=1024
)

In [49]:
def create_ansatz(params, num_qubits):
    
    qc = QuantumCircuit(num_qubits)
    qc.ry(params[0], 0)
    
    return qc

In [50]:
potential = "QHO"
cutoff = 16 
H_matrix = calculate_Hamiltonian(cutoff, potential)
num_qubits = int(1 + np.log2(cutoff))

hamiltonian = SparsePauliOp.from_operator(H_matrix)

In [51]:
params = [0]
qc = create_ansatz(params, num_qubits)

pm = generate_preset_pass_manager(optimization_level=3)
isa_circuit = pm.run(qc)
isa_observable = hamiltonian.apply_layout(isa_circuit.layout)

In [64]:
estimator = Estimator(backend=simulator)
estimator.options.default_shots = 1024

C:\Users\Johnk\AppData\Local\Temp\ipykernel_14572\666957122.py:1: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  estimator = Estimator(backend=simulator)


In [67]:
job = estimator.run([(isa_circuit, isa_observable)])

In [68]:
job.result()

PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.float64(0.13427413546164207)), metadata={'target_precision': 0.03125, 'shots': 1024, 'circuit_metadata': {}})], metadata={'version': 2})

: 